# Advanced Exercises

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from photutils.detection import DAOStarFinder
from astropy.wcs import WCS
from astroquery.vizier import Vizier

from astropy.table import Table
from astropy.units import Quantity
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
from astropy import units as u

from photutils.aperture import CircularAperture
from photutils.aperture import CircularAnnulus
from photutils.aperture import aperture_photometry

from astropy.visualization import (imshow_norm, 
                                   ZScaleInterval,
                                   SquaredStretch,
                                   LinearStretch,
                                  LogStretch,
                                   SqrtStretch,
                                  MinMaxInterval,
                                  ImageNormalize)

from astropy.convolution import AiryDisk2DKernel

from photutils import centroid_com, centroid_1dg, centroid_2dg

# Astropy photometry examples

### First let's explore some of the documentation for astropy's photometry functions. I'll leave you with a few examples but it will be your job to figure out how to get to the end!

Let's start by adding a point source to a field of Poisson noise.

In [ ]:
source = np.array(AiryDisk2DKernel(5,x_size=31,y_size=31)) # 2D gaussian star
source = source/np.sum(source)*1000.0  # Normalize the total flux of the source to 1000

image = np.zeros([101,101])
image[50:81,10:41] = source  #y,x, note y starts from top

image += 2.0 # Add sky background of 2 e-
image = np.random.poisson(image)  # Sample the image with Poisson noise
image = image + np.random.normal(0.0,1.0,[101,101])  # Add a read noise of 1 e-

plt.imshow(image,vmin=0,vmax=0.5*np.max(image), origin='lower')
plt.colorbar()

Next we will find the intensity-weighted centroid of this image using astropy photutils. This is easy since there's only one source in the field.

In [ ]:
x, y = centroid_1dg(image)
print(x, y)

Using DAOStarFinder is useful when there are multiple sources in the image. Let's add another one. 

https://photutils.readthedocs.io/en/stable/api/photutils.detection.DAOStarFinder.html

In [ ]:
image2 = np.zeros([101,101])
image2[50:81,10:41] = source  #y,x, note y starts from top

source2 = np.array(AiryDisk2DKernel(5,x_size=25,y_size=25))
source2 = source2/np.sum(source2) * 500
image2[20:45,45:70] = source2

image2 += 2.0 # Add sky background of 2 e-
image2 = np.random.poisson(image2)  # Sample the image with Poisson noise
image2 = image2 + np.random.normal(0.0,1.0,[101,101])  # Add a read noise of 1 e-

plt.imshow(image2,vmin=0,vmax=0.5*np.max(image2), origin='lower')
plt.colorbar()
plt.show()

We again define the background with sigma clipped stats, then run the detection algorithm.

In [ ]:
mean, median, std = sigma_clipped_stats(image2, sigma=3.0)
print(mean, median, std)

In [ ]:
daofind = DAOStarFinder(fwhm=5.0, threshold=5.*std)  
source_find = daofind(image2-median)

source_find


Next we will perform aperture photometry. The first step is to define an aperture in which we will measure flux. Then we have to characterize the background flux in an annulus that does not touch the source. The background flux per pixel is the measured background divided by the area of the annulus, which we will subtract from the aperture sum to get the background-subtracted flux.

https://photutils.readthedocs.io/en/stable/aperture.html

Note that DAOStarFinder reports a flux *density* in the flux column of the table which is a function of area and the defined threshold. It's more useful to work in total fluxes when measuring colors, which we can do directly with photutil's aperture photometry.

In [ ]:
# For our single source

positions = (x, y) # Set the centroid of our source using the most accurate value from above
apertures = CircularAperture(positions, r=5.)  # Use an aperture with r=5 pixels (~FWHM)

# Now define an annulus - r_in and r_out set position of annulus
annulus_apertures = CircularAnnulus(positions, r_in=8., r_out=10.)

# We can combine the apertures into a tuple. Aperture_sum_0 is science, aperture_sum_1 is bg
apers=[apertures,annulus_apertures]  
phot_table = aperture_photometry(image, apers)
print(phot_table)

# Now calculate background per pixel
bkg_mean = phot_table['aperture_sum_1'][0] / annulus_apertures.area
bkg_sum = bkg_mean * apertures.area
print(bkg_sum)

# Now we have our background subtracted flux!
final_sum = phot_table['aperture_sum_0'][0] - bkg_sum
print(final_sum)

# We are missing an aperture correction to get from the aperture flux to infinite flux,
# but don't worry about that right now :) 

In [ ]:
# For an array of sources

positions2 = np.array([source_find['xcentroid'], source_find['ycentroid']]).T # Set the centroid of our source using the most accurate value from above
apertures2 = CircularAperture(positions2, r=5.)  # Use an aperture with r=5 pixels (~FWHM)

# Now define an annulus - r_in and r_out set position of annulus
annulus_apertures2 = CircularAnnulus(positions2, r_in=8., r_out=10.)

# We can combine the apertures into a tuple. Aperture_sum_0 is science, aperture_sum_1 is bg
apers2=[apertures2,annulus_apertures2]  
phot_table2 = aperture_photometry(image2, apers2)
print(phot_table2)

# Now calculate background per pixel
bkg_mean2 = phot_table2['aperture_sum_1'] / annulus_apertures2.area
bkg_sum2 = bkg_mean2 * apertures2.area

# Now we have our background subtracted flux!
final_sum_source1 = phot_table2['aperture_sum_0'][0] - bkg_sum2[0]
final_sum_source2 = phot_table2['aperture_sum_0'][1] - bkg_sum2[1]

print(final_sum_source1, final_sum_source2)

plt.imshow(image2,vmin=0,vmax=0.5*np.max(image), origin='lower')
apertures2.plot(color='black')
annulus_apertures2.plot(color='black', linestyle='--')
plt.colorbar()
plt.show()


Then we search for sources above a 5-sigma detection threshold and subtract the background.

# Exercise 1: Producing a color-magnitude diagram of an open star cluster 

by Jackie Champagne (thank you to Erica Sawczynec for your help!)

Let's take a look at an open star cluster with B and V filter images taken at the Las Cumbres Observatory. This is NGC 2509, in the Puppis constellation.

## Goals: 
- Practice reading in FITS files
- Perform object detection algorithm to find stars
- Measure photometry
- Calculate colors and produce a color-magnitude diagram
- Bonus: learn how to manually zeropoint data

To perform photometry, we want to use a source-finding algorithm which detects circular objects with a certain significance threshold. To do this, we will need to calculate the statistics of the image to characterize the background.

### 1. Open the fits files here. You have B, V, and R images of NGC 2509.

#### Hints: 1) Recall how to access different attributes of the HDU object. 2) If you aren't sure what the files are all called, you can try using glob to find all FITS files in a directory:

from glob import glob

fitslist = glob('*.fits')

print(fitslist)

In [ ]:
# open files here

### 2. Measure the mean, median, and standard deviation of the images, clipped at 3 sigma. Use sigma_clipped_stats from astropy.

Sigma clipping helps to throw out the tails of a Gaussian distribution, since real sources will be far brighter than 3sigma.

#### Hint: SIGMA_CLIPPED_STATS outputs a tuple containing the mean, median, and std, so you can define the output as three variables here, i.e.: mean, median, std = sigma_clipped_stats(arguments here)

In [ ]:
# your stats here

### 3. Use the DAOStarFinder to detect stars in both images with a FWHM of 5 pixels at a threshold of 5sigma. 

In [ ]:
# find stars in B

In [ ]:
# find stars in V

### 4. Use the positions (xcentroid, ycentroid) from your daofind outputs and perform aperture photometry using astropy photutils. Display the images and plot the apertures on top. Finally, convert your fluxes to magnitudes using

$$ m = -2.5 * log_{10}F $$

In [ ]:
# A helpful function to display a color image nicely

def show_fits(ax, hdu, interval=ZScaleInterval(), stretch=LinearStretch(), cmap='gray'):
    im, norm = imshow_norm(hdu.data, ax,
                           interval=interval,
                           stretch=stretch,
                           cmap=cmap,
                        origin='lower')
    ax.set_xlabel('X coord')
    ax.set_ylabel('Y coord')
    return ax, im

# Define your apertures here

# Then plot image 
fig, ax = plt.subplots(1, 1, figsize = (10,10))
ims, norma = imshow_norm(Bdata, ax, origin='lower',  interval=ZScaleInterval(), 
                                             cmap='cool', 
                                           stretch=SquaredStretch())
# This is how you plot apertures
apertures_B.plot(color='black', lw=1.5, alpha=0.5)



In [ ]:
# Repeat for V


In [ ]:
# Perform photometry and convert to magnitude

### 5. Before we create a CMD we need to make sure we are measuring photometry for the same stars in each image. 

Using astropy.wcs, grab the WCS information from the headers. https://docs.astropy.org/en/stable/wcs/wcsapi.html#basic-usage
There is a function that allows you to convert x, y positions to world coordinates. Do that!

Then, create a SkyCoord object from your B and V catalogs. You can learn more about that here: https://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html

Finally, use match_to_catalog_sky to match it to the V catalog. Make sure to match it within 0.5 arcseconds so we know it's the same source. You can learn more about that here: https://docs.astropy.org/en/stable/coordinates/matchsep.html#matching-catalogs


In [ ]:
# First access the WCS information from the header using WCS() from astropy.wcs... make sense?

# Then convert the x and y positions of the stars to RA and Dec. Which function should you use?

# Now match your catalogs following the documentation above



### 6. Plot your CMD! On the y axis, a CMD has magnitudes, like B, and the x axis has a color, which is the difference between two magnitudes, like B-V. 

In [ ]:
# your CMD here

## This doesn't have a lot of physical significance, because the images are not physically calibrated: the magnitudes are instrumental rather than absolute. Physical calibration involves finding the *zeropoint* of an image, which relates instrumental magnitudes from one instrument to accepted standard magnitudes. 

## To do this, we normally query a catalog to find calibrated magnitudes taken with the same filter on a different telescope. The US Naval Observatory catalog has B and V magnitudes that we can calibrate to. First, let's use the sky positions of the stars you found in your LCO images using the WCS (world coordinate system) information. Matching to a known catalog also helps confirm that your sources are real and not noise spikes.


### 7. Using Vizier, query the USNO database in a 5 arcmin region around your stars; you will need the WCS information as you used above.

### 8. Match your star catalog to the USNO results using astropy SkyCoord's match_to_catalog_sky like you did before.

Documentation you will need:

astroquery: https://astroquery.readthedocs.io/en/latest/vizier/vizier.html


In [ ]:
# Collect all known stars in the USNO observatory within 10 arcmin
v = Vizier()
results = ????
print(results[0].keys())

# Define USNO positions with SkyCoord


# And match your catalog to 0.2 arcsec precision using match_to_catalog_sky
                       

### Finally, plot your measured B magnitudes against the USNO B magnitudes. The median of the difference in magnitudes is the scalar offset to be added to all of your measured B mags. The last step is to plot your calibrated CMD!

In [ ]:
# Your zeropoint plot here; plot the median difference using plt.axhline

In [ ]:
# Your CMD here

# CHALLENGE EXERCISE!

## Say we didn't have a similar filter to zeropoint our images. Let's brainstorm some ideas for how we might calibrate our images!


### Goals: Query an optical database to find physically-calibrated magnitudes for these stars. Using your photometric models from Photometry1, how would you figure out the correct B and V magnitudes using colors from another library?
